In [2]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding

import numpy as np
import matplotlib.pyplot as plt

class SnakeEnv(gym.Env):
    metadata = { 'render.modes': ['human'] }
    
    def __init__(self, board_shape=(12,12), start_pos=):
        self.board_shape = board_shape
        self.board = np.zeros(board_shape)
        self.board[:,0] = 1
        self.board[:,-1] = 1
        self.board[0,:] = 1
        self.board[-1,:] = 1
        self.board
        self.info = {}
        self._reset()
        
    def _reset(self):
        start

In [9]:
board_shape = (12, 12)

# -1: food, 0: empty, 1: snake, 2: wall
board = np.zeros(board_shape, dtype=int)
board[:,0]  = 2
board[:,-1] = 2
board[0,:]  = 2
board[-1,:] = 2

In [10]:
print(board)

[[2 2 2 2 2 2 2 2 2 2 2 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 0 0 0 0 0 0 0 0 2]
 [2 2 2 2 2 2 2 2 2 2 2 2]]


In [60]:
class Board:
    def __init__(self, shape=(12,12)):
        self.shape = shape
        self.board = np.zeros(self.shape, dtype=int)
        
        self._prepare_borders()
        
    def _prepare_borders(self):
        self.board[:,0]  = 2
        self.board[:,-1] = 2
        self.board[0,:]  = 2
        self.board[-1,:] = 2
        
    def show(self):
        for row in self.board:
            print(row)

class Snake:
    def __init__(self, board, starting_position=None, starting_orientation=3, starting_length=3):
        self.board = board
        if starting_position is None:
            starting_position = np.array(list(map(lambda x: x//2, board.shape)))
        self.position = starting_position # 2-tuple of ints
        self.orientation = starting_orientation # 0: down, 1: left, 2: up, 3: right
        
        converter = {
            0: np.array([1, 0]),
            1: np.array([0, -1]),
            2: np.array([-1, 0]),
            3: np.array([0, 1]),
        }
        
        self.body = [self.position - 2*converter[self.orientation], 
                     self.position - converter[self.orientation], 
                     self.position]
        
    def put_on_board(self):
        for pos in self.body:
            self.board.board[pos[0], pos[1]] = 1
        
        

In [61]:
board = Board((12, 12))

In [62]:
board.show()

[2 2 2 2 2 2 2 2 2 2 2 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 2 2 2 2 2 2 2 2 2 2 2]


In [63]:
snake = Snake(board)

In [66]:
snake.put_on_board()

board.show()

[2 2 2 2 2 2 2 2 2 2 2 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 1 1 1 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 0 0 0 0 0 0 0 0 0 0 2]
[2 2 2 2 2 2 2 2 2 2 2 2]
